In [2]:
import pandas as pd
import re

## Hazardous Liquid Pipeline Accident Reports for 1986 - 2001 

In [11]:
file = "sources_from_links/PHMSA_Pipeline_Safety_Flagged_Incidents/hl1986to2001.xlsx"

data = pd.read_excel(file, sheetname="hl1986to2001")

df = pd.DataFrame(data=data, columns=['IDATE','ACSTATE', 'CAUSE',
                                      'NAME', 'NARRATIVE', 'TOTAL_COST'])

df.head(10)

,IDATE,ACSTATE,CAUSE,NAME,NARRATIVE,TOTAL_COST
0,1986-01-06,CO,OUTSIDE FORCE DAMAGE,WYCO PIPE LINE CO,NaN,57000.0
1,1986-01-13,TX,OTHER,SEMINOLE PIPELINE CO,NaN,1500.0
2,1986-01-30,TX,CORROSION,MOBIL PIPELINE CO,CRUDE OIL WAS DISCOVERED LEAKING FROM THE BOTT...,0.0
3,1986-01-15,TX,OUTSIDE FORCE DAMAGE,MOBIL PIPELINE CO,A CONTRACTOR WAS GRADING A LOGGING ROAD FOR TH...,0.0
4,1986-02-07,TX,CORROSION,SHELL PIPE LINE CORP,NaN,0.0
5,1986-01-30,TX,CORROSION,SHELL PIPE LINE CORP,NaN,0.0
6,1986-01-31,NJ,OTHER,SUN PIPELINE CO,IT APPEARS PIPE WAS DAMAGED BY THIRD PARTY. UN...,0.0
7,1986-02-07,TX,INCORRECT OPERATION BY OPERATOR PERSONNEL,PHILLIPS PIPE LINE CO,LINE WAS ACCIDENTLY BLOCKED IN AND LINE FAILED...,0.0
8,1986-01-30,KS,OUTSIDE FORCE DAMAGE,JAYHAWK PIPELINE CORP,"APPROXIMATELY 1045 JANUARY 30, 1986 A CONTRACT...",0.0
9,1986-02-10,MN,FAILED WELD,WOOD RIVER PIPELINE CO,"INSTALLED CROSSOVER VALVES FROM 18"" TO 16"" LIN...",3000.0


In [12]:
df.CAUSE.value_counts()

OTHER                                         831
OUTSIDE FORCE DAMAGE                          783
CORROSION                                     772
INCORRECT OPERATION BY OPERATOR PERSONNEL     197
FAILED PIPE                                   170
MALFUNCTION OF CONTROL OR RELIEF EQUIPMENT    150
FAILED WELD                                   141
Name: CAUSE, dtype: int64

In [13]:
df['CAUSE'][df.CAUSE == "OTHER"] = 'other'
df['CAUSE'][df.CAUSE == "OUTSIDE FORCE DAMAGE"] = 'other'
df['CAUSE'][df.CAUSE == "CORROSION"] = 'maintenance'
df['CAUSE'][df.CAUSE == "INCORRECT OPERATION BY OPERATOR PERSONNEL"] = 'technical failure'
# df['CAUSE'][df.CAUSE == "ACCIDENTALLY CAUSED BY OPERATOR"] = 'sabotage'
df['CAUSE'][df.CAUSE == "FAILED PIPE"] = 'leaks and spills'
df['CAUSE'][df.CAUSE == "MALFUNCTION OF CONTROL OR RELIEF EQUIPMENT"] = 'maintenance'
df['CAUSE'][df.CAUSE == "FAILED WELD"] = 'leaks and spills'

# df['CAUSE'][df.CAUSE == "NATURAL FORCES"] = 'natural disasters'


# df['CAUSE'][df.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'



/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can 

In [14]:
df.rename(columns = {'IDATE'  :'Incident Date',
                     'NAME'   : 'Name of Owner',
                     'ACSTATE': 'State',
                     'CAUSE'   : 'Cause of Loss', 
                     'NARRATIVE'  : 'Text',
                     'TOTAL_COST'  : 'Total Gas Released ($)'},
                      inplace = True)


In [15]:
df.insert(3, column='Name of Installation', value=data.ACCOUNTY.values.astype(str))

df['Name of Installation'] = 'UNKNOWN (' + df['Name of Installation'] + ' COUNTY)'

# df['Incident Date'] =  pd.to_datetime(df['Incident Date'], format='%m/%d/%Y')

df['Total Gas Released ($)'] = df['Total Gas Released ($)'].fillna(0).astype(int)

df['Sources'] = "https://www.phmsa.dot.gov/pipeline/library/data-stats/flagged-data-files"

df.head(10)

,Incident Date,State,Cause of Loss,Name of Installation,Name of Owner,Text,Total Gas Released ($),Sources
0,1986-01-06,CO,other,UNKNOWN (EL PASO COUNTY),WYCO PIPE LINE CO,NaN,57000,https://www.phmsa.dot.gov/pipeline/library/dat...
1,1986-01-13,TX,other,UNKNOWN (CONCHO COUNTY),SEMINOLE PIPELINE CO,NaN,1500,https://www.phmsa.dot.gov/pipeline/library/dat...
2,1986-01-30,TX,maintenance,UNKNOWN (CALDWELL COUNTY),MOBIL PIPELINE CO,CRUDE OIL WAS DISCOVERED LEAKING FROM THE BOTT...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
3,1986-01-15,TX,other,UNKNOWN (JASPER COUNTY),MOBIL PIPELINE CO,A CONTRACTOR WAS GRADING A LOGGING ROAD FOR TH...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
4,1986-02-07,TX,maintenance,UNKNOWN ( COUNTY),SHELL PIPE LINE CORP,NaN,0,https://www.phmsa.dot.gov/pipeline/library/dat...
5,1986-01-30,TX,maintenance,UNKNOWN (FORT BEND COUNTY),SHELL PIPE LINE CORP,NaN,0,https://www.phmsa.dot.gov/pipeline/library/dat...
6,1986-01-31,NJ,other,UNKNOWN (MERCER COUNTY),SUN PIPELINE CO,IT APPEARS PIPE WAS DAMAGED BY THIRD PARTY. UN...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
7,1986-02-07,TX,technical failure,UNKNOWN (HUTCHINSON COUNTY),PHILLIPS PIPE LINE CO,LINE WAS ACCIDENTLY BLOCKED IN AND LINE FAILED...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
8,1986-01-30,KS,other,UNKNOWN (HASKELL COUNTY),JAYHAWK PIPELINE CORP,"APPROXIMATELY 1045 JANUARY 30, 1986 A CONTRACT...",0,https://www.phmsa.dot.gov/pipeline/library/dat...
9,1986-02-10,MN,leaks and spills,UNKNOWN (DAKOTA COUNTY),WOOD RIVER PIPELINE CO,"INSTALLED CROSSOVER VALVES FROM 18"" TO 16"" LIN...",3000,https://www.phmsa.dot.gov/pipeline/library/dat...


In [16]:
writer = pd.ExcelWriter('hazardous_liquid_pipeline_incident_1986_to_2001.xlsx')
df.to_excel(writer,'hl1986to2001')
writer.save()

## Hazardous Liquid Pipeline Accident Reports for 2002 - 2009

In [17]:
file2 = "sources_from_links/PHMSA_Pipeline_Safety_Flagged_Incidents/hl2002to2009.xlsx"

data2 = pd.read_excel(file2, sheetname="hl2002to2009")

df2 = pd.DataFrame(data=data2, columns=['IDATE','OPSTATE', 'CAUSE',
                                      'NAME', 'NARRATIVE', 'TOTAL_COST'])
df2.head()

,IDATE,OPSTATE,CAUSE,NAME,NARRATIVE,TOTAL_COST
0,2002-01-23,TX,MATERIAL AND/OR WELD FAILURES,"TE PRODUCTS PIPELINE CO., LP",NaN,510
1,2002-01-27,TX,OTHER,"EQUISTAR CHEMICALS, L.P.",REGARDING PART C5: SECTIONS OF THIS LINE WERE...,98242
2,2002-01-24,TX,INCORRECT OPERATION,MOBIL PIPELINE CO,"ON JANUARY 24,2002 MOBIL PIPELINE COMPANY INCU...",47670
3,2002-01-07,TX,EQUIPMENT,CHEVRON PIPELINE CO,NaN,3000
4,2002-01-23,TX,EQUIPMENT,"KINDER MORGAN ENERGY PARTNERS, L.P.",NaN,2662


In [19]:
l = df2.CAUSE.value_counts()
l

EQUIPMENT                        1041
CORROSION                         692
MATERIAL AND/OR WELD FAILURES     381
INCORRECT OPERATION               322
EXCAVATION DAMAGE                 183
OTHER                             173
NATURAL FORCES                    162
OTHER OUTSIDE FORCE DAMAGE         75
Name: CAUSE, dtype: int64

In [33]:
df2['CAUSE'][df2.CAUSE == "OTHER OUTSIDE FORCE DAMAGE"] = 'other'
df2['CAUSE'][df2.CAUSE == "EXCAVATION DAMAGE"] = 'sabotage'
df2['CAUSE'][df2.CAUSE == "OTHER"] = 'other'
df2['CAUSE'][df2.CAUSE == "NATURAL FORCES"] = 'natural disasters'
df2['CAUSE'][df2.CAUSE == "MATERIAL AND/OR WELD FAILURES"] = 'leaks and spills'
df2['CAUSE'][df2.CAUSE == "INCORRECT OPERATION"] = 'technical failure'

df2['CAUSE'][df2.CAUSE == "EQUIPMENT"] = 'maintenance'
df2['CAUSE'][df2.CAUSE == "CORROSION"] = 'maintenance'



# df2['CAUSE'][df2.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'



/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can 

In [34]:
df2.rename(columns = {'IDATE'  :'Incident Date',
                     'NAME'   : 'Name of Owner',
                     'OPSTATE': 'State',
                     'CAUSE'   : 'Cause of Loss', 
                     'NARRATIVE'  : 'Text',
                     'TOTAL_COST'  : 'Total Gas Released ($)'},
                      inplace = True)


In [35]:
df2.insert(3, column='Name of Installation', value=data2.OPCOUNTY.values.astype(str))

df2['Name of Installation'] = 'UNKNOWN (' + df2['Name of Installation'] + ' COUNTY )'

# df2['Incident Date'] =  pd.to_datetime(df2['Incident Date'], format='%m/%d/%Y')

df2['Total Gas Released ($)'] = df2['Total Gas Released ($)'].fillna(0).astype(int)

df2['Sources'] = "https://www.phmsa.dot.gov/pipeline/library/data-stats/flagged-data-files"

df2.head()

,Incident Date,State,Cause of Loss,Name of Installation,Name of Owner,Text,Total Gas Released ($),Sources
0,2002-01-23,TX,leaks and spills,UNKNOWN (HARRIS COUNTY ),"TE PRODUCTS PIPELINE CO., LP",NaN,510,https://www.phmsa.dot.gov/pipeline/library/dat...
1,2002-01-27,TX,other,UNKNOWN (BRAZORIA COUNTY ),"EQUISTAR CHEMICALS, L.P.",REGARDING PART C5: SECTIONS OF THIS LINE WERE...,98242,https://www.phmsa.dot.gov/pipeline/library/dat...
2,2002-01-24,TX,technical failure,UNKNOWN (HARRIS COUNTY ),MOBIL PIPELINE CO,"ON JANUARY 24,2002 MOBIL PIPELINE COMPANY INCU...",47670,https://www.phmsa.dot.gov/pipeline/library/dat...
3,2002-01-07,TX,maintenance,UNKNOWN (HARRIS COUNTY ),CHEVRON PIPELINE CO,NaN,3000,https://www.phmsa.dot.gov/pipeline/library/dat...
4,2002-01-23,TX,maintenance,UNKNOWN (HARRIS COUNTY ),"KINDER MORGAN ENERGY PARTNERS, L.P.",NaN,2662,https://www.phmsa.dot.gov/pipeline/library/dat...


In [36]:
writer = pd.ExcelWriter('hazardous_liquid_pipeline_incident_2002_to_2009.xlsx')
df2.to_excel(writer,'hl2002to2009')
writer.save()

## Hazardous Liquid Pipeline Accident Reports for 2010 - Present

In [38]:
file3 = "sources_from_links/PHMSA_Pipeline_Safety_Flagged_Incidents/hl2010toPresent.xlsx"

data3 = pd.read_excel(file3, sheetname="hl2010toPresent")

df3 = pd.DataFrame(data=data3, columns=['LOCAL_DATETIME','ONSHORE_STATE_ABBREVIATION', 'CAUSE','NAME', 'NARRATIVE', 'TOTAL_COST'])

df3.head(10)

,LOCAL_DATETIME,ONSHORE_STATE_ABBREVIATION,CAUSE,NAME,NARRATIVE,TOTAL_COST
0,2010-02-16 07:42:00,IL,EQUIPMENT FAILURE,"MAGELLAN PIPELINE COMPANY, LP",NaN,30515
1,2010-03-01 11:50:00,KS,OTHER INCIDENT CAUSE,"CHAPARRAL ENERGY, LLC","AT 11:50 AM ON THE MORNING OF MARCH 1, 2010 A ...",3510
2,2010-02-22 10:38:00,TX,CORROSION FAILURE,LDH ENERGY PIPELINE L.P.,LEAK DISCOVERED BY CONTRACTOR WORKING FOR ANOT...,26860
3,2010-02-19 06:50:00,NJ,EQUIPMENT FAILURE,"BUCKEYE PARTNERS, LP","AT 6:50 AM ON FEBRUARY 19, 2010, BUCKEYE PERSO...",17100
4,2010-02-21 12:45:00,TX,CORROSION FAILURE,"PLAINS PIPELINE, L.P.",INTERNAL CORROSION ON 10-INCH PIPELINE RESULTE...,17485
5,2010-02-22 10:56:00,WA,INCORRECT OPERATION,CONOCOPHILLIPS,"ON FEBRUARY 22, 2010 OPERATOR LINED UP YPL 10:...",8790
6,2010-02-23 23:30:00,IA,INCORRECT OPERATION,ONEOK NGL PIPELINE LP,WHILE ATTEMPTING TO MAKE UNLEADED GASOLINE DEL...,129007
7,2010-02-25 04:50:00,OK,INCORRECT OPERATION,ONEOK NGL PIPELINE LP,OPERATOR WAS ISOLATING PIPELINE GATHERING SYST...,1599786
8,2010-02-20 06:30:00,TX,EQUIPMENT FAILURE,SUNOCO PIPELINE L.P.,FAILURE OF AN AIR ELIMINATOR RESULTED IN OIL F...,93513
9,2010-02-25 10:00:00,NC,EQUIPMENT FAILURE,COLONIAL PIPELINE CO,ROUTINE STATION CHECK ON 02/25/10 DISCOVERED S...,15950


In [39]:
df3.CAUSE.value_counts()

EQUIPMENT FAILURE                   1347
CORROSION FAILURE                    636
INCORRECT OPERATION                  413
MATERIAL FAILURE OF PIPE OR WELD     221
NATURAL FORCE DAMAGE                 127
OTHER INCIDENT CAUSE                 114
EXCAVATION DAMAGE                    108
OTHER OUTSIDE FORCE DAMAGE            60
Name: CAUSE, dtype: int64

In [40]:
df3['CAUSE'][df3.CAUSE == "EXCAVATION DAMAGE"] = 'sabotage'

df3['CAUSE'][df3.CAUSE == "OTHER OUTSIDE FORCE DAMAGE"] = 'other'
df3['CAUSE'][df3.CAUSE == "OTHER INCIDENT CAUSE"] = 'other'
df3['CAUSE'][df3.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'
df3['CAUSE'][df3.CAUSE == "MATERIAL FAILURE OF PIPE OR WELD"] = 'leaks and spills'
df3['CAUSE'][df3.CAUSE == "INCORRECT OPERATION"] = 'technical failure'
df3['CAUSE'][df3.CAUSE == "CORROSION FAILURE"] = 'maintenance'

df3['CAUSE'][df3.CAUSE == "EQUIPMENT FAILURE"] = 'maintenance'



# df3['CAUSE'][df3.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'



/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [41]:
df3.rename(columns = {'LOCAL_DATETIME'  :'Incident Date',
                     'NAME'   : 'Name of Owner',
                     'ONSHORE_STATE_ABBREVIATION': 'State',
                     'CAUSE'   : 'Cause of Loss', 
                     'NARRATIVE'  : 'Text',
                     'TOTAL_COST'  : 'Total Gass/Property Loss ($)'},
                      inplace = True)


In [42]:
df3.insert(3, column='Name of Installation', value=data3.ONSHORE_COUNTY_NAME.values.astype(str))

df3['Name of Installation'] = 'UNKNOWN (' + df3['Name of Installation'] + ' COUNTY)'

# df3['Incident Date'] =  pd.to_datetime(df3['Incident Date'], format='%Y%m%d')
# df3['Name of Installation'] = df3['Name of Installation'].fillna('UNKNOWN').astype(str)
df3['Total Gass/Property Loss ($)'] = df3['Total Gass/Property Loss ($)'].fillna(0).astype(int)

df3['Sources'] = "https://www.phmsa.dot.gov/pipeline/library/data-stats/flagged-data-files"

df3.head(10)

,Incident Date,State,Cause of Loss,Name of Installation,Name of Owner,Text,Total Gass/Property Loss ($),Sources
0,2010-02-16 07:42:00,IL,maintenance,UNKNOWN (DUPAGE COUNTY),"MAGELLAN PIPELINE COMPANY, LP",NaN,30515,https://www.phmsa.dot.gov/pipeline/library/dat...
1,2010-03-01 11:50:00,KS,other,UNKNOWN (SEWARD COUNTY),"CHAPARRAL ENERGY, LLC","AT 11:50 AM ON THE MORNING OF MARCH 1, 2010 A ...",3510,https://www.phmsa.dot.gov/pipeline/library/dat...
2,2010-02-22 10:38:00,TX,maintenance,UNKNOWN (HOWARD COUNTY),LDH ENERGY PIPELINE L.P.,LEAK DISCOVERED BY CONTRACTOR WORKING FOR ANOT...,26860,https://www.phmsa.dot.gov/pipeline/library/dat...
3,2010-02-19 06:50:00,NJ,maintenance,UNKNOWN (UNION COUNTY),"BUCKEYE PARTNERS, LP","AT 6:50 AM ON FEBRUARY 19, 2010, BUCKEYE PERSO...",17100,https://www.phmsa.dot.gov/pipeline/library/dat...
4,2010-02-21 12:45:00,TX,maintenance,UNKNOWN (UPTON COUNTY),"PLAINS PIPELINE, L.P.",INTERNAL CORROSION ON 10-INCH PIPELINE RESULTE...,17485,https://www.phmsa.dot.gov/pipeline/library/dat...
5,2010-02-22 10:56:00,WA,technical failure,UNKNOWN (SPOKANE COUNTY),CONOCOPHILLIPS,"ON FEBRUARY 22, 2010 OPERATOR LINED UP YPL 10:...",8790,https://www.phmsa.dot.gov/pipeline/library/dat...
6,2010-02-23 23:30:00,IA,technical failure,UNKNOWN (POTTAWATAMIE COUNTY),ONEOK NGL PIPELINE LP,WHILE ATTEMPTING TO MAKE UNLEADED GASOLINE DEL...,129007,https://www.phmsa.dot.gov/pipeline/library/dat...
7,2010-02-25 04:50:00,OK,technical failure,UNKNOWN (GRANT COUNTY),ONEOK NGL PIPELINE LP,OPERATOR WAS ISOLATING PIPELINE GATHERING SYST...,1599786,https://www.phmsa.dot.gov/pipeline/library/dat...
8,2010-02-20 06:30:00,TX,maintenance,UNKNOWN (GREGG COUNTY),SUNOCO PIPELINE L.P.,FAILURE OF AN AIR ELIMINATOR RESULTED IN OIL F...,93513,https://www.phmsa.dot.gov/pipeline/library/dat...
9,2010-02-25 10:00:00,NC,maintenance,UNKNOWN (nan COUNTY),COLONIAL PIPELINE CO,ROUTINE STATION CHECK ON 02/25/10 DISCOVERED S...,15950,https://www.phmsa.dot.gov/pipeline/library/dat...


In [44]:
writer = pd.ExcelWriter('hazardous_liquid_pipeline_incident_2010_to_Present.xlsx')
df3.to_excel(writer,'hl2010toPresent')
writer.save()

## Gas Transmission and Gathering Systems Incident Reports for 1986 - 2001

In [87]:
file4 = "sources_from_links/PHMSA_Pipeline_Safety_Flagged_Incidents/gtgg1986to2001.xlsx"

data4 = pd.read_excel(file4, sheetname="gtgg1986to2001")

df4 = pd.DataFrame(data=data4, columns=['IDATE','ACSTATE', 'CAUSE','NAME', 'NARRATIVE', 'TOTAL_COST'])

df4.head(10)

,IDATE,ACSTATE,CAUSE,NAME,NARRATIVE,TOTAL_COST
0,1986-01-27,WY,OTHER,PHILLIPS 66 NATURAL GAS CO,PIPELINE PRESSURE WAS BEING REDUCED BY VENTING...,0
1,1986-01-17,IL,DAMAGE BY OUTSIDE FORCE,PANHANDLE EASTERN PIPELINE CO,THE OPERATION OF A BACKHOE IN THE VICINITY OF ...,60000
2,1986-01-15,LA,DAMAGE BY OUTSIDE FORCE,COLUMBIA GULF TRANSMISSION CO,NaN,120000
3,1986-02-10,AL,CONSTRUCTION/MATERIAL DEFECT,FLORIDA GAS TRANSMISSION CO,CRACK ON TOP OF GIRTH WELD WAS LEAKING.,0
4,1986-02-21,TX,OTHER,TRANSWESTERN PIPELINE CO,A BROKEN BOLT IN THE COMPRESSOR CAUSED VIBRATI...,117000
5,1986-02-21,KY,OTHER,TEXAS EASTERN GAS PIPELINE CO,THE PIPELINE RUPTURED NEAR THE SOUTH END OF A ...,1300000
6,1986-02-14,KS,DAMAGE BY OUTSIDE FORCE,KN ENERGY INC (KANSAS NEBRASKA NATURAL GAS CO ...,LEAK REPORT COVERS A NATURAL GAS LEAK THAT OCC...,0
7,1986-02-13,WV,OTHER,CONSOLIDATED GAS TRANSMISSION CORP,"STATION EMPLOYEE OPENED 1"" BALL VALVE ON BYPAS...",600
8,1986-02-19,NV,DAMAGE BY OUTSIDE FORCE,SOUTHWEST GAS CORP,"FLOODING OF THE TRUCKEE RIVER NEAR WADSWORTH, ...",0
9,1986-02-24,AR,CONSTRUCTION/MATERIAL DEFECT,ARKANSAS LOUISIANA GAS CO (ARKLA GAS CO),"LINE ""S"", 20"" O.D. PIPELINE BLEW OUT AT 75' SE...",147354


In [88]:
df4.CAUSE.value_counts()

DAMAGE BY OUTSIDE FORCE         511
CORROSION                       304
OTHER                           295
CONSTRUCTION/MATERIAL DEFECT    178
Name: CAUSE, dtype: int64

In [89]:
df4['CAUSE'][df4.CAUSE == "DAMAGE BY OUTSIDE FORCE"] = 'other'
df4['CAUSE'][df4.CAUSE == "CORROSION"] = 'leaks and spills'
df4['CAUSE'][df4.CAUSE == "OTHER"] = 'other'
# df4['CAUSE'][df4.CAUSE == "NATURAL FORCES"] = 'natural disasters'
df4['CAUSE'][df4.CAUSE == "CONSTRUCTION/MATERIAL DEFECT"] = 'maintenance'
# df4['CAUSE'][df4.CAUSE == "INCORRECT OPERATION"] = 'technical failure'
df4['CAUSE'][df4.CAUSE == "EQUIPMENT"] = 'maintenance'
# df4['CAUSE'][df4.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'


/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can 

In [90]:
df4.rename(columns = {'IDATE'  :'Incident Date',
                     'NAME'   : 'Name of Owner',
                     'ACSTATE': 'State',
                     'CAUSE'   : 'Cause of Loss', 
                     'NARRATIVE'  : 'Text',
                     'TOTAL_COST'  : 'Total Gass/Property Loss ($)'},
                      inplace = True)


df4.insert(3, column='Name of Installation', value=data4.ACCOUNTY.values.astype(str))

df4['Name of Installation'] = 'UNKNOWN (' + df4['Name of Installation'] + ' COUNTY)'

# df4['Incident Date'] =  pd.to_datetime(df4['Incident Date'], format='%Y%m%d')
# df4['Name of Installation'] = df4['Name of Installation'].fillna('UNKNOWN').astype(str)
df4['Total Gass/Property Loss ($)'] = df4['Total Gass/Property Loss ($)'].fillna(0).astype(int)

df4['Sources'] = "https://www.phmsa.dot.gov/pipeline/library/data-stats/flagged-data-files"

df4.head(10)

,Incident Date,State,Cause of Loss,Name of Installation,Name of Owner,Text,Total Gass/Property Loss ($),Sources
0,1986-01-27,WY,other,UNKNOWN (CAMPBELL COUNTY),PHILLIPS 66 NATURAL GAS CO,PIPELINE PRESSURE WAS BEING REDUCED BY VENTING...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
1,1986-01-17,IL,other,UNKNOWN (MORGAN COUNTY),PANHANDLE EASTERN PIPELINE CO,THE OPERATION OF A BACKHOE IN THE VICINITY OF ...,60000,https://www.phmsa.dot.gov/pipeline/library/dat...
2,1986-01-15,LA,other,UNKNOWN (OFFSHORE COUNTY),COLUMBIA GULF TRANSMISSION CO,NaN,120000,https://www.phmsa.dot.gov/pipeline/library/dat...
3,1986-02-10,AL,maintenance,UNKNOWN (BALDWIN COUNTY),FLORIDA GAS TRANSMISSION CO,CRACK ON TOP OF GIRTH WELD WAS LEAKING.,0,https://www.phmsa.dot.gov/pipeline/library/dat...
4,1986-02-21,TX,other,UNKNOWN (HARRIS COUNTY),TRANSWESTERN PIPELINE CO,A BROKEN BOLT IN THE COMPRESSOR CAUSED VIBRATI...,117000,https://www.phmsa.dot.gov/pipeline/library/dat...
5,1986-02-21,KY,other,UNKNOWN (GARRARD COUNTY),TEXAS EASTERN GAS PIPELINE CO,THE PIPELINE RUPTURED NEAR THE SOUTH END OF A ...,1300000,https://www.phmsa.dot.gov/pipeline/library/dat...
6,1986-02-14,KS,other,UNKNOWN (WALLACE COUNTY),KN ENERGY INC (KANSAS NEBRASKA NATURAL GAS CO ...,LEAK REPORT COVERS A NATURAL GAS LEAK THAT OCC...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
7,1986-02-13,WV,other,UNKNOWN (LEWIS COUNTY),CONSOLIDATED GAS TRANSMISSION CORP,"STATION EMPLOYEE OPENED 1"" BALL VALVE ON BYPAS...",600,https://www.phmsa.dot.gov/pipeline/library/dat...
8,1986-02-19,NV,other,UNKNOWN ( COUNTY),SOUTHWEST GAS CORP,"FLOODING OF THE TRUCKEE RIVER NEAR WADSWORTH, ...",0,https://www.phmsa.dot.gov/pipeline/library/dat...
9,1986-02-24,AR,maintenance,UNKNOWN (NEVADA COUNTY),ARKANSAS LOUISIANA GAS CO (ARKLA GAS CO),"LINE ""S"", 20"" O.D. PIPELINE BLEW OUT AT 75' SE...",147354,https://www.phmsa.dot.gov/pipeline/library/dat...


In [91]:
writer = pd.ExcelWriter('gas_transmission_gathering_1986_2001.xlsx')
df4.to_excel(writer,'gtgg1986to2001')
writer.save()

## Gas Transmission and Gathering Systems Incident Reports for 2002 - 2009

In [82]:
file5 = "sources_from_links/PHMSA_Pipeline_Safety_Flagged_Incidents/gtgg2002to2009.xlsx"

data5 = pd.read_excel(file5, sheetname="gtgg2002to2009")

df5 = pd.DataFrame(data=data5, columns=['IDATE','ACSTATE', 'CAUSE','NAME', 'NARRATIVE', 'TOTAL_COST'])

df5.head(5)

,IDATE,ACSTATE,CAUSE,NAME,NARRATIVE,TOTAL_COST
0,2002-01-26,NaN,CORROSION,NORTHERN NATURAL GAS CO (ENRON),NaN,1300000
1,2002-03-06,KY,MATERIAL AND/OR WELD FAILURES,TENNESSEE GAS PIPELINE CO,THERE WAS A RUPTURE IN THE COMPRESSOR STATION ...,2011000
2,2002-03-13,LA,CORROSION,WILLIAMS FIELD SERVICES,WHILE TRANSPORTING OPERATIONAL PERSONNEL BACK ...,190000
3,2002-02-28,NaN,OTHER,EL PASO FIELD SERVICES,GARLAND GASPARD OF EL PASO FIELD SERVICES WAS ...,171342
4,2002-03-04,MD,MATERIAL AND/OR WELD FAILURES,WILLIAMS GAS PIPELINE - TRANSCO,COMPANY EMPLOYEE WHILE MOWING ROW FOUND GAS ES...,350000


In [83]:
df5.CAUSE.value_counts()

CORROSION                        267
EXCAVATION DAMAGE                150
MATERIAL AND/OR WELD FAILURES    150
NATURAL FORCES                   148
OTHER                            108
OTHER OUTSIDE FORCE DAMAGE        93
EQUIPMENT                         86
INCORRECT OPERATION               27
Name: CAUSE, dtype: int64

In [84]:
df5['CAUSE'][df5.CAUSE == "CORROSION"] = 'leaks and spills'
df5['CAUSE'][df5.CAUSE == "EXCAVATION DAMAGE"] = 'sabotage'
df5['CAUSE'][df5.CAUSE == "MATERIAL AND/OR WELD FAILURES"] = 'maintenance'
df5['CAUSE'][df5.CAUSE == "NATURAL FORCES"] = 'natural disasters'
df5['CAUSE'][df5.CAUSE == "OTHER"] = 'other'
df5['CAUSE'][df5.CAUSE == "OTHER OUTSIDE FORCE DAMAGE"] = 'other'
df5['CAUSE'][df5.CAUSE == "EQUIPMENT"] = 'maintenance'
df5['CAUSE'][df5.CAUSE == "INCORRECT OPERATION"] = 'technical failure'
# df5['CAUSE'][df5.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'


/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can 

In [85]:
df5.rename(columns = {'IDATE'  :'Incident Date',
                     'NAME'   : 'Name of Owner',
                     'ACSTATE': 'State',
                     'CAUSE'   : 'Cause of Loss', 
                     'NARRATIVE'  : 'Text',
                     'TOTAL_COST'  : 'Total Gass/Property Loss ($)'},
                      inplace = True)


df5.insert(3, column='Name of Installation', value=data5.ACCOUNTY.values.astype(str))

df5['Name of Installation'] = 'UNKNOWN (' + df5['Name of Installation'] + ' COUNTY)'

# df5['Incident Date'] =  pd.to_datetime(df5['Incident Date'], format='%Y%m%d')
# df5['Name of Installation'] = df5['Name of Installation'].fillna('UNKNOWN').astype(str)
df5['Total Gass/Property Loss ($)'] = df5['Total Gass/Property Loss ($)'].fillna(0).astype(int)

df5['Sources'] = "https://www.phmsa.dot.gov/pipeline/library/data-stats/flagged-data-files"

df5.head(10)

,Incident Date,State,Cause of Loss,Name of Installation,Name of Owner,Text,Total Gass/Property Loss ($),Sources
0,2002-01-26,NaN,leaks and spills,UNKNOWN (nan COUNTY),NORTHERN NATURAL GAS CO (ENRON),NaN,1300000,https://www.phmsa.dot.gov/pipeline/library/dat...
1,2002-03-06,KY,maintenance,UNKNOWN (MONTGOMERY COUNTY),TENNESSEE GAS PIPELINE CO,THERE WAS A RUPTURE IN THE COMPRESSOR STATION ...,2011000,https://www.phmsa.dot.gov/pipeline/library/dat...
2,2002-03-13,LA,leaks and spills,UNKNOWN (TERREBONNE COUNTY),WILLIAMS FIELD SERVICES,WHILE TRANSPORTING OPERATIONAL PERSONNEL BACK ...,190000,https://www.phmsa.dot.gov/pipeline/library/dat...
3,2002-02-28,NaN,other,UNKNOWN (nan COUNTY),EL PASO FIELD SERVICES,GARLAND GASPARD OF EL PASO FIELD SERVICES WAS ...,171342,https://www.phmsa.dot.gov/pipeline/library/dat...
4,2002-03-04,MD,maintenance,UNKNOWN (HOWARD COUNTY),WILLIAMS GAS PIPELINE - TRANSCO,COMPANY EMPLOYEE WHILE MOWING ROW FOUND GAS ES...,350000,https://www.phmsa.dot.gov/pipeline/library/dat...
5,2002-03-15,MI,leaks and spills,UNKNOWN (IRON COUNTY),GREAT LAKES GAS TRANSMISSION CO,"UPON COMPLETION OF THE FAILURE INVESTIGATION, ...",3104629,https://www.phmsa.dot.gov/pipeline/library/dat...
6,2002-03-18,IL,sabotage,UNKNOWN (COOK COUNTY),NATURAL GAS PIPELINE CO OF AMERICA (KMI),NaN,1810,https://www.phmsa.dot.gov/pipeline/library/dat...
7,2002-02-22,NE,other,UNKNOWN (DAKOTA COUNTY),NORTHERN NATURAL GAS CO (ENRON),LEAKING GAS WAS REPORTED BY A CONTRACTOR WHO W...,73000,https://www.phmsa.dot.gov/pipeline/library/dat...
8,2002-03-26,WY,maintenance,UNKNOWN (CARBON COUNTY),COLORADO INTERSTATE GAS CO,A LEAK WAS DISCOVERED DURING F 1 LEAK SURVEY O...,145000,https://www.phmsa.dot.gov/pipeline/library/dat...
9,2002-03-26,OH,leaks and spills,UNKNOWN (JACKSON COUNTY),TENNESSEE GAS PIPELINE CO,THERE WAS A LEAK NEAR MP 202-4 + 14.0. WHILE ...,70000,https://www.phmsa.dot.gov/pipeline/library/dat...


In [86]:
writer = pd.ExcelWriter('gas_transmission_gathering_2002_to_2009.xlsx')
df5.to_excel(writer,'gtgg2002to2009')
writer.save()

## GAS TRANSMISSION AND GATHERING PIPELINE SYSTEMS 2010 - Present

In [125]:
file6 = "sources_from_links/PHMSA_Pipeline_Safety_Flagged_Incidents/gtggungs2010toPresent.xlsx"

data6 = pd.read_excel(file6, sheetname="gtggungs2010toPresent")

df6 = pd.DataFrame(data=data6, columns=['LOCAL_DATETIME','OPERATOR_STATE_ABBREVIATION', 'CAUSE','NAME', 'NARRATIVE', 'EST_COST_GAS_RELEASED'])
df6.head(5)

,LOCAL_DATETIME,OPERATOR_STATE_ABBREVIATION,CAUSE,NAME,NARRATIVE,EST_COST_GAS_RELEASED
0,2010-02-09 01:32:00,OK,NATURAL FORCE DAMAGE,CENTERPOINT ENERGY GAS TRANSMISSION,THE REGULATOR FOR THE PNEUMATIC ACTUATOR FROZE...,172714.0
1,2010-01-06 04:32:00,AL,CORROSION FAILURE,SOUTHERN NATURAL GAS CO,ON JANUARY 6 2010 SOUTHERN NATURAL GAS (SNG) C...,242699.0
2,2010-02-16 08:18:00,OK,EQUIPMENT FAILURE,CENTERPOINT ENERGY GAS TRANSMISSION,"AT APPROXIMATELY 08:18 HRS ON FEBRUARY 22, A F...",6429.0
3,2010-02-17 04:40:00,TX,EQUIPMENT FAILURE,ENTERPRISE PRODUCTS OPERATING LLC,"ON FEBRUARY 17, 2010 THE WISE COUNTY COMPRESSO...",189000.0
4,2010-02-19 10:03:00,OK,EQUIPMENT FAILURE,ENOGEX LLC,AT 10:03 AM AN I&E TECHNICIAN WAS WORKING INSI...,10850.0


In [126]:
df6.CAUSE.value_counts()

EQUIPMENT FAILURE                   262
CORROSION FAILURE                   196
EXCAVATION DAMAGE                   120
MATERIAL FAILURE OF PIPE OR WELD    109
NATURAL FORCE DAMAGE                 70
OTHER INCIDENT CAUSE                 58
OTHER OUTSIDE FORCE DAMAGE           57
INCORRECT OPERATION                  48
Name: CAUSE, dtype: int64

In [127]:
df6['CAUSE'][df6.CAUSE == "EQUIPMENT FAILURE"] = 'maintenance'

df6['CAUSE'][df6.CAUSE == "CORROSION FAILURE"] = 'leaks and spills'
df6['CAUSE'][df6.CAUSE == "EXCAVATION DAMAGE"] = 'sabotage'
df6['CAUSE'][df6.CAUSE == "MATERIAL FAILURE OF PIPE OR WELD"] = 'maintenance'
df6['CAUSE'][df6.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'
df6['CAUSE'][df6.CAUSE == "OTHER INCIDENT CAUSE"] = 'other'
df6['CAUSE'][df6.CAUSE == "OTHER OUTSIDE FORCE DAMAGE"] = 'other'
df6['CAUSE'][df6.CAUSE == "INCORRECT OPERATION"] = 'technical failure'
# df6['CAUSE'][df6.CAUSE == "NATURAL FORCE DAMAGE"] = 'natural disasters'


/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [128]:
df6.rename(columns = {'LOCAL_DATETIME'  :'Incident Date',
                     'NAME'   : 'Name of Owner',
                     'OPERATOR_STATE_ABBREVIATION': 'State',
                     'CAUSE'   : 'Cause of Loss', 
                     'NARRATIVE'  : 'Text',
                     'EST_COST_GAS_RELEASED'  : 'Total Gass/Property Loss ($)'},
                      inplace = True)


df6.insert(3, column='Name of Installation', value=data6.ONSHORE_COUNTY_NAME.values.astype(str))

df6['Name of Installation'] = 'UNKNOWN (' + df6['Name of Installation'] + ' COUNTY)'

# df6['Incident Date'] =  pd.to_datetime(df6['Incident Date'], format='%Y%m%d')
# df6['Name of Installation'] = df6['Name of Installation'].fillna('UNKNOWN').astype(str)
df6['Total Gass/Property Loss ($)'] = df6['Total Gass/Property Loss ($)'].fillna(0).astype(int)

df6['Sources'] = "https://www.phmsa.dot.gov/pipeline/library/data-stats/flagged-data-files"

df6.head(10)

,Incident Date,State,Cause of Loss,Name of Installation,Name of Owner,Text,Total Gass/Property Loss ($),Sources
0,2010-02-09 01:32:00,OK,natural disasters,UNKNOWN (CUSTER COUNTY),CENTERPOINT ENERGY GAS TRANSMISSION,THE REGULATOR FOR THE PNEUMATIC ACTUATOR FROZE...,172714,https://www.phmsa.dot.gov/pipeline/library/dat...
1,2010-01-06 04:32:00,AL,leaks and spills,UNKNOWN (WINSTON COUNTY),SOUTHERN NATURAL GAS CO,ON JANUARY 6 2010 SOUTHERN NATURAL GAS (SNG) C...,242699,https://www.phmsa.dot.gov/pipeline/library/dat...
2,2010-02-16 08:18:00,OK,maintenance,UNKNOWN (ROFER MILLS COUNTY COUNTY),CENTERPOINT ENERGY GAS TRANSMISSION,"AT APPROXIMATELY 08:18 HRS ON FEBRUARY 22, A F...",6429,https://www.phmsa.dot.gov/pipeline/library/dat...
3,2010-02-17 04:40:00,TX,maintenance,UNKNOWN (WISE COUNTY),ENTERPRISE PRODUCTS OPERATING LLC,"ON FEBRUARY 17, 2010 THE WISE COUNTY COMPRESSO...",189000,https://www.phmsa.dot.gov/pipeline/library/dat...
4,2010-02-19 10:03:00,OK,maintenance,UNKNOWN (OKLAHOMA COUNTY),ENOGEX LLC,AT 10:03 AM AN I&E TECHNICIAN WAS WORKING INSI...,10850,https://www.phmsa.dot.gov/pipeline/library/dat...
5,2010-03-02 04:20:00,KY,maintenance,UNKNOWN (RENO COUNTY),"SOUTHERN STAR CENTRAL GAS PIPELINE, INC",AT 4:20AM THE GAS CONTROLLER NOTICED A PRESSUR...,313880,https://www.phmsa.dot.gov/pipeline/library/dat...
6,2010-03-05 12:30:00,CA,other,UNKNOWN (SOLANO COUNTY),PACIFIC GAS & ELECTRIC CO,A FIRE OCCURRED AT APPROXIMATELY 1230 HOURS ON...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
7,2010-03-01 14:00:00,TX,maintenance,UNKNOWN (MOJAVE COUNTY),EL PASO NATURAL GAS CO,AN ANOMALY INDICATION FROM AN ILI RUN WAS BEIN...,0,https://www.phmsa.dot.gov/pipeline/library/dat...
8,2010-03-06 15:30:00,AL,other,UNKNOWN (nan COUNTY),TENNESSEE GAS PIPELINE CO (EL PASO),"ON MARCH 6, 2010, A 15 FEET DIAMETER GAS BUBBL...",0,https://www.phmsa.dot.gov/pipeline/library/dat...
9,2010-01-21 12:29:00,TX,maintenance,UNKNOWN (COPIAH COUNTY),"SOUTHEAST SUPPLY HEADER, LLC","ON JANUARY 21, 2010, A SOUTHERN PINES ELECTRIC...",1310,https://www.phmsa.dot.gov/pipeline/library/dat...


In [129]:
writer = pd.ExcelWriter('gas_transmission_gathering_2010_to_Present.xlsx')
df6.to_excel(writer,'gtggungs2010toPresent')
writer.save()